In [1]:
from Modele_db.modele_db import *
from Pianki.Analiza_pianek.funkcje_analizy_pianek import *

# pd.set_option()

In [2]:
Podsumowanie_paczek_i_Pw("24/54")

OBIĘTOSC PACZEK (Z OWATAMI)
23/01: 211m3
24/01: 182m3
WST: 86m3
Objetość wszystkich paczek: 480m3
PW 24/54 spakowano: 63M3


In [3]:
br = Braki(WOLNE="SALDO")[0] #USUNAC PROBLEM Z DUBLUJACYMI SIE ZAMOWIANIAMI(JEDNO DOSTARCZONE DRUGIE W DRODZE TWORZĄ SIĘ DWIE POZYCJE)
br.sort_values(by=["GRUPA", "OPIS"])
# br.to_excel("BRAKI.xlsx")

,OPIS,WOLNE_SALDO,SALDO,PACZKA,DATA_KOMPLETACJI,ZAMOWIONE,UWAGI,GRUPA
1,DUO P82x82,-1.0,0.0,23/01,2024-08-12,0.0,...,1
3,RITZ [L,-1.0,0.0,WST,2024-08-26,0.0,...,1
4,SAMOA P100x100,-2.0,0.0,23/01,2024-08-12,0.0,...,1
12,UNO P63x63,-1.0,3.0,WST,2024-08-26,0.0,...,1
8,WIL 4 SIED Z BOCZ - OWA,-13.0,1.0,23/01,2024-08-12,0.0,...,1
0,DIVA LA,-2.0,0.0,23/01,2024-08-12,40.0,"PIANKI ZAMOWIONE P, , ,",2
11,KELLY [1],-9.0,3.0,23/01,2024-08-12,60.0,"PIANKI ZAMOWIONE P, , ,",2
9,LENOX [1o],-169.0,71.0,23/01,2024-08-12,220.0,"PIANKI ZAMOWIONE P, , ,",2
2,ONYX ][,-12.0,10.0,24/01,2024-08-19,80.0,"PIANKI ZAMOWIONE P, , ,",2
5,STONE 3,-2.0,11.0,24/01,2024-08-19,40.0,"PIANKI ZAMOWIONE P, , ,",2


In [4]:
Zagrozone(WOLNE="SALDO")
zgr = Zagrozone(False,WOLNE="SALDO")
# zgr
# zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)]
# zgr[(zgr.SALDO < zgr.MIN)]
zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)&(zgr.SALDO < zgr.MIN)]

WOLNE PONIZEJ MIN: 99 POZYCJE
WOLNE PONIZEJ MIN NIE ZAMOWIONE: 66 POZYCJE
SALDO PONIZEJ MIN: 60 POZYCJE
SALDO PONIZEJ MIN NIE ZAMOWIONE: 35 POZYCJE


,OPIS,ZAMOWIONE,CZEKA_NA_SPAKOWANIE,SALDO,MIN,WOLNE_SALDO
0,AVANT [1],0.0,0.0,0.0,2,0.0
7,AVANT 2,0.0,0.0,2.0,3,2.0
8,"AVANT 2,5",0.0,0.0,2.0,5,1.0
19,AVANT LM][,0.0,0.0,7.0,15,4.0
20,AVANT P104x64,0.0,0.0,7.0,10,5.0
21,AVANT P112x84,0.0,0.0,9.0,12,8.0
22,AVANT P112x97,0.0,0.0,20.0,21,18.0
34,"AMALFI 1,5 MEMORY",0.0,0.0,20.0,30,16.0
40,AMALFI [LA MEMORY,0.0,0.0,6.0,8,6.0
50,CALYPSO E80D,0.0,0.0,2.0,10,1.0


In [5]:
Ogolna_analiza_objetosci()
print("---\nZAPEŁNIENIE MAGAZYNU")
Ogolna_analiza_objetosci("podsum_prc")

MAX_obj:                    3250
ZAMOWIONE_obj:              390
CZEKA_NA_SPAKOWANIE_obj:    269
SALDO_obj:                  2035
WOLNE_obj:                  1555
WOLNE_NIE_SPAK_obj:         1824
DO_ZAM_obj:                 1131
---
ZAPEŁNIENIE MAGAZYNU
SALDO / MAX: 62.6%
WOLNE / MAX: 47.8%
WOLNE_NIE_SPAK / MAX: 56.1%
ZAPEŁNIENIE MAG PO ZDJECIU 2 PACZEK RAZEM Z ZAMOWIENIAMI: 68.1%


In [6]:
#IMPORT ANAMLIZY PIANEK
from Pianki.Analiza_pianek.instrukcje_zamawiana import instrukcja_zamawiania_pianpol as izp, instrukcja_zamawiania_ciech as izc
from Pianki.Analiza_pianek.Podsumowanie_analizy_pianek import Podsumowanie_analizy_pianek

arp = Podsumowanie_analizy_pianek(izp)
# arc = Podsumowanie_analizy_pianek(izc)



In [7]:
arp

OBJ_CIECH       40.80
OBJ_VITA        84.69
OBJ_PIANPOL    627.88
RAZEM          753.37

In [8]:
(arp > 1).query("GRUPA < 4")

,GRUPA,MODEL,BRYL_DO_ZAMOWIENIA,WSPL_DO_ZAM,OBJ_CIECH,OBJ_VITA,OBJ_PIANPOL,*WSPL_ZAP_WOLNE,*WSPL_ZAP_ZAM
1,1,ELIXIR,172.0,43.0,0.0,0.00,69.06,54.0,57.0
0,1,AVANT,318.0,38.0,0.0,0.00,82.48,66.0,66.0
3,1,HORIZON,344.0,31.0,0.0,84.69,17.11,41.0,69.0
2,1,STONE,163.0,21.0,0.0,0.00,55.90,35.0,82.0
5,2,DIVA,110.0,48.0,0.0,0.00,37.71,30.0,60.0
8,2,WILLOW,147.0,39.0,0.0,0.00,58.20,30.0,61.0
4,2,AMALFI,191.0,36.0,0.0,0.00,72.19,65.0,65.0
9,2,REVERSO,212.0,32.0,0.0,0.00,58.45,48.0,68.0
6,2,MAXWELL,41.0,24.0,0.0,0.00,14.64,66.0,81.0
7,2,OXYGEN,36.0,16.0,0.0,0.00,9.92,90.0,90.0


In [9]:
arp.Tabela_podsumowania_analizy.query("GRUPA == 4")

,GRUPA,MODEL,BRYL_DO_ZAMOWIENIA,WSPL_DO_ZAM,OBJ_CIECH,OBJ_VITA,OBJ_PIANPOL,*WSPL_ZAP_WOLNE,*WSPL_ZAP_ZAM
23,4,GREY,146.0,97.0,0.0,0.0,10.67,3.0,3.0
18,4,COCO,137.0,86.0,0.0,0.0,24.68,14.0,14.0
21,4,UNO,61.0,76.0,0.0,0.0,2.21,26.0,26.0
19,4,OVAL,175.0,50.0,0.0,0.0,20.43,21.0,50.0
20,4,LENOX,104.0,40.0,0.0,0.0,11.84,-73.0,60.0
22,4,KELLY,29.0,36.0,0.0,0.0,2.03,-11.0,64.0


In [ ]:
arp["AVANT"].ar

In [ ]:
"sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db"



In [10]:
Zapis_danych_do_Archiwum(2432, analiza, arp._Tabela_podsumowania_analizy, "sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db")